In [1]:
!pip install unidecode
!pip install lightgbm==2.2.3 --user

     |████████████████████████████████| 1.2 MB 574 kB/s 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings
import lightgbm as lgb
from itertools import combinations
from unidecode import unidecode
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, StratifiedKFold
warnings.filterwarnings('ignore')
#pd.options.display.max_columns = 50
plt.style.use('ggplot')

%matplotlib inline

In [3]:
df_train = pd.read_csv('../input/klps-creditscring-challenge-for-students/train.csv')
df_test = pd.read_csv('../input/klps-creditscring-challenge-for-students/test.csv')


**Preprocessing time columns**

In [4]:
date_time = ["Field_{}".format(i) for i in [1, 2, 43, 44]]
date = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33, 35, 40]]
other_date = ["F_startDate", "F_endDate", "E_startDate", "E_endDate", "C_startDate",
              "C_endDate", "G_startDate", "G_endDate", "A_startDate", "A_endDate"
             ]
time = date + date_time + other_date

def split_date_train(s):
    try:
        date = s.split('-')
        return date
    except:
        return np.nan
def split_date_test(s):
    try:
        date = s.split('/')
        return date
    except:
        return np.nan

df_train_c = df_train.copy()
df_test_c =  df_test.copy()
for col in date:
    df_train_c[col] = df_train_c[col].apply(split_date_train)
for col in date:
    df_test_c[col] = df_test_c[col].apply(split_date_test)

new_col = []
for col in date: 
    df_train_c[col+'year'] =[s[0] if type(s) == list else np.nan  for s in df_train_c[col]]
    df_train_c[col+'month'] =[s[1]if type(s) == list and len(s) >1  else np.nan for s in df_train_c[col]]
    df_train_c[col+'day'] =[s[2]if type(s) == list and len(s) > 2  else np.nan for s in df_train_c[col]]
    new_col.append(col+'year')
    new_col.append(col+'month')
    new_col.append(col+'day')
    
# the columns in "date" are the ones with format year/month/day in train set   

#new_col_test = []
for col in date: 
    df_test_c[col+'year'] =[s[2]if type(s) == list and len(s) > 2  else np.nan for s in df_test_c[col]]
    df_test_c[col+'month'] =[s[0] if type(s) == list else np.nan  for s in df_test_c[col]]
    df_test_c[col+'day'] =[s[1]if type(s) == list and len(s) >1  else np.nan for s in df_test_c[col]]
    #new_col_test.append(col+'year')
    #new_col_test.append(col+'month')
    #new_col_test.append(col+'day')
    
# the columns in "date" are the ones with format month/day/year in test set  


df_train_c = df_train_c.drop(date,1)
df_test_c = df_test_c.drop(date,1)
# and then check if the columns are match between train and test

list(df_train_c)[2:] == list(df_test_c)[1:]

def get_date(s):
    try:
        return s[:10]
    except:
        return np.nan
    
for col in date_time:
    df_train_c[col] = df_train_c[col].apply(get_date)
    df_test_c[col] = df_test_c[col].apply(get_date)
# the columns in date_time at the format year/month/day in both train/test set
for col in date_time:
    df_train_c[col+'year']=[s[:4]if type(s) == str else np.nan for s in df_train_c[col]]
    
    df_train_c[col+'month']=[s[5:7]if type(s) == str else np.nan for s in df_train_c[col]]
    
    df_train_c[col+'day']=[s[8:10] if type(s) == str else np.nan for s in df_train_c[col]]

new_col_1 = []

for col in date_time:
    df_test_c[col+'year']=[s[:4]if type(s) == str else np.nan for s in df_test_c[col]]
    
    df_test_c[col+'month']=[s[5:7]if type(s) == str else np.nan for s in df_test_c[col]]
    
    df_test_c[col+'day']=[s[8:10] if type(s) == str else np.nan for s in df_test_c[col]]
    new_col_1.append(col+'year')
    new_col_1.append(col+'month')
    new_col_1.append(col+'day')


df_train_c = df_train_c.drop(date_time,1)

df_test_c = df_test_c.drop(date_time,1)

#check again if the columns are match

list(df_train_c)[2:] == list(df_test_c)[1:]

for col in other_date:
    df_train_c[col] = df_train_c[col].apply(split_date_train)
for col in other_date: 
    df_train_c[col+'year'] =[s[0] if type(s) == list else np.nan  for s in df_train_c[col]]
    df_train_c[col+'month'] =[s[1]if type(s) == list and len(s) >1  else np.nan for s in df_train_c[col]]
    df_train_c[col+'day'] =[s[2]if type(s) == list and len(s) > 2  else np.nan for s in df_train_c[col]]
    
    
# this the data in other_date in format year-month-day in train set

for col in other_date:
    df_test_c[col] = df_test_c[col].apply(split_date_test)
new_col_2 = []
for col in other_date: 
    df_test_c[col+'year'] =[s[2]if type(s) == list and len(s) > 2  else np.nan for s in df_test_c[col]]
    df_test_c[col+'month'] =[s[0] if type(s) == list else np.nan  for s in df_test_c[col]]
    df_test_c[col+'day'] =[s[1]if type(s) == list and len(s) >1  else np.nan for s in df_test_c[col]]
    new_col_2.append(col+'year')
    new_col_2.append(col+'month')
    new_col_2.append(col+'day')
    
# the columns in "date" are the ones with format month/day/year in test set   

df_train_c = df_train_c.drop(other_date,1)
df_test_c = df_test_c.drop(other_date,1)

list(df_train_c)[2:] == list(df_test_c)[1:]

#convert to string first
df_train_c["ngaySinh"]= df_train_c["ngaySinh"].apply(str)
df_test_c["ngaySinh"] = df_test_c["ngaySinh"].apply(str)

def get_age(s):
    try:
        return 2020 - int(s[:4])
    except:
        return np.nan
    
df_train_c["ngaySinh"]= df_train_c["ngaySinh"].apply(get_age)
df_test_c["ngaySinh"] = df_test_c["ngaySinh"].apply(get_age)
        
#process "Field_34"
df_train_c["Field_34"]= df_train_c["Field_34"].apply(str)
df_test_c["Field_34"] = df_test_c["Field_34"].apply(str)
def get_year_34(s):
  try:
    return (s[:4])
  except:
    return np.nan
def get_month_34(s):
  try:
    return (s[4:6])
  except:
    return np.nan

df_train_c["Field_34year"]= df_train_c["Field_34"].apply(get_year_34)
df_train_c["Field_34month"] = df_train_c["Field_34"].apply(get_month_34)

df_test_c["Field_34year"]= df_test_c["Field_34"].apply(get_year_34)
df_test_c["Field_34month"] = df_test_c["Field_34"].apply(get_month_34)

df_train_c = df_train_c.drop("Field_34",1)

df_test_c = df_test_c.drop("Field_34",1)

list(df_train_c)[2:] == list(df_test_c)[1:]


True

**Preprocessing other columns**

In [5]:
#the list of province in Vietnam
province_list = ['An Giang','Bà Rịa – Vũng Tàu','Bắc Giang','Bắc Kạn','Bạc Liêu','Bắc Ninh','Bến Tre',
'Bình Định','Bình Dương','Bình Phước','Bình Thuận','Cà Mau','Cần Thơ','Cao Bằng','Đà Nẵng','Đắk Lắk',
'Đắk Nông','Điện Biên','Đồng Nai','Đồng Tháp','Gia Lai','Hà Giang','Hà Nam','Hà Nội','Hà Tĩnh','Hải Dương',
'Hải Phòng','Hậu Giang','Hòa Bình','Hưng Yên','Khánh Hòa','Kiên Giang','Kon Tum','Lai Châu','Lâm Đồng',
'Lạng Sơn','Lào Cai','Long An','Nam Định','Nghệ An','Ninh Bình','Ninh Thuận','Phú Thọ','Phú Yên',
'Quảng Bình','Quảng Nam','Quảng Ngãi','Quảng Ninh','Quảng Trị','Sóc Trăng','Sơn La','Tây Ninh',
'Thái Bình','Thái Nguyên','Thanh Hóa','Huế','Tiền Giang','Hồ Chí Minh','Trà Vinh','Tuyên Quang','Vĩnh Long',
'Vĩnh Phúc','Yên Bái' ]
#make all lower case
province_list = [i.lower() for i in province_list]

field_46 = pd.read_csv("../input/field-46-groupcsv/field_46_group.csv")
field_46 = field_46.replace(np.nan, '@#', regex=True)
for i in list(field_46):
  field_46[i] = field_46[i].str.lower()
#fucntion to correct diachi
def diachi(s):
  for i in province_list:
    try:
      if i in s:
        return i
    except:
      pass
  return s

# fucntion to correct Field_46

def group_46(s):
  for col in list(field_46):
    for i in set(field_46[col]):
      try:
        if i in s:
          return col 
      except:
        pass 
  return "None"

df_fe = df_train_c.drop(['label'], 1).append(df_test_c)
df_fe['Field_46'] = df_fe['Field_46'].str.lower()
df_fe['Field_46'] = df_fe['Field_46'].apply(group_46)


#Field_49
df_fe['Field_49'] = df_fe['Field_49'].str.lower()
df_fe['Field_49'] = df_fe['Field_49'] .apply(diachi)
def group_hcm(s):
  district_num = [ 'q.1','q.2','q.3','q.4','q.5','q.6','q.7','q.8','q.9'
                   ,'q.10','q.11','q.12',
                   'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10','q11','q12','q.']
  name = ['hcm', 'saigon','sài gòn','quận','tpcm']
  district_str = ['tân phú','phú nhuận','thủ đức','gò vấp',
                  'bình thạnh','bình tân','củ chi','hóc môn'
                  ,'bình chánh','nhà bè','cần giờ','tân bình']
  hcm_city = district_num + name + district_str
  for i in hcm_city:
    try:
      if i in s:
        return 'hồ chí minh'
    except:
      pass
  return s

df_fe['Field_49'] = df_fe['Field_49'].apply(group_hcm)


# replace all the rest with nan
def replace_more(s):
  for i in province_list:
    try:
      if i in s: 
        return s         
    except:
      pass
  return "None"
  

df_fe['Field_49'] = df_fe['Field_49'].apply(replace_more)

df_fe['diaChi'] = df_fe['diaChi'].str.lower()
df_fe['diaChi'] = df_fe['diaChi'].apply(diachi)
df_fe['diaChi'] = df_fe['diaChi'].apply(group_hcm)
df_fe['diaChi'] = df_fe['diaChi'].apply(replace_more)


df_fe['Field_56'] = df_fe['Field_56'].str.lower()
from collections import Counter
counter_56 = Counter(df_fe['Field_56'])
dict_56 = { "ngoài quốc doanh": ["ngoài quốc doanh"],
           
            "nhà nước":["nhà nước",'đoàn thể',"lực lượng vũ trang","đại biểu"],
           
           "doanh nghiệp":["doanh nghiệp"], 
           
           "ngoài công lập":["ngoài công lập"],
           
           "hành chánh": ["hành chánh",'sự nghiệp quận',"sự nghiệp"],
           
           "xã": ['phường','xã', "hợp tác xã"],
           
           "nghèo": ["nghèo","thất nghiệp","trợ cấp","khó khăn","bảo trợ"], 
           
           "hộ gia đình": ["hộ gia đình", "sinh viên", "cá thể","thân nhân","đối tượng"],
           
           "others": ["nước ngoài"]
           
           
          }

def group_56(s):
    for col in list(dict_56):
        for i in dict_56[col]:
            try:
                if i in s:
                    return col
            except:
                pass
    return "None"

df_fe["Field_56"] = df_fe["Field_56"].apply(group_56)


dict_61 = { "ngoài quốc doanh": ["ngoài quốc doanh"], "hộ gia đình":["hộ gia đình","cán bộ","cá thể"],  
           
           "ngoài công lập": ["ngoài công lập"], "nghèo": ["nghèo", "thất nghiệp", "dân tộc","khó khăn","trợ cấp"], 
           
           "nước ngoài": ["dtnn"],  "nhà nước": ["đại biểu","nhà nước", "đảng"],
           
           "others": ["sinh viên","thân nhân","người","phường xã"]
           
           
               

          }

def group_61(s):
    for col in list(dict_61):
        for i in dict_61[col]:
            try:
                if i in s:
                    return col
            except:
                pass
    return "None"   


df_fe['Field_61'] = df_fe['Field_61'].str.lower()
df_fe["Field_61"] = df_fe["Field_61"].apply(group_61)


**More preprocessing other columns**

In [6]:
def str_normalize(s):
    s = str(s).strip().lower()
    s = re.sub(' +', " ", s)
    return s

def process_location(df):
    for col in ["currentLocationLocationId", "homeTownLocationId", "currentLocationLatitude", "currentLocationLongitude", 
                   "homeTownLatitude", "homeTownLongitude"]:
        df[col].replace(0, np.nan, inplace=True)

    df["currentLocationLocationId"] = df["currentLocationLocationId"].apply(str_normalize).astype("category")
    df["homeTownLocationId"] = df["homeTownLocationId"].apply(str_normalize).astype("category")

    return df


def job_category(x):
    if type(x) == str:
        if "công nhân" in x or "cnv" in x or "cn" in x or "may công nghiệp" in x or "lao động" in x\
        or "thợ" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x or "c.n" in x or "lđ" in x:
            return "CN"
        elif "giáo viên" in x or "gv" in x or "gíao viên" in x:
            return "GV"
        elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x or "cb" in x or "nhõn viờn" in x:
            return "NV"
        elif "tài xế" in x or "lái" in x or "tài xê" in x:
            return "TX"
        elif "quản lý" in x or "phó phòng" in x or "hiệu phó" in x:
            return "QL"
        elif "undefined" in x:
            return "missing"
        elif "giám đốc" in x or "hiệu trưởng" in x:
            return "GĐ"
        elif "phục vụ" in x:
            return "PV"
        elif "chuyên viên" in x:
            return  "CV"
        elif "bác sĩ" in x or "dược sĩ" in x or "y sĩ" in x or "y sỹ" in x:
            return "BS"
        elif "y tá" in x:
            return "YT"
        elif "hộ sinh" in x:
            return "HS"
        elif "chủ tịch" in x:
            return "CT"
        elif "bếp" in x:
            return "ĐB"
        elif "sư" in x:
            return "KS"
        elif "dưỡng" in x:
            return "ĐD"
        elif "kỹ thuật" in x or "kĩ thuật" in x:
            return "KTV"
        elif "diễn viên" in x:
            return "DV"
        else:
            return "missing"
    else:
        return x    
    
def process_diaChi_maCv(df):
    df["maCv"] = df["maCv"].apply(str_normalize).apply(job_category).astype("category")
    return df


def combine_gender(s):
    x, y = s
    
    if x != x and y != y:
        return "nan"
    
    if x != x:
        return y.lower()
    
    return x.lower()

def process_gender(df):
    df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
    return df


def process_misc(df):        
    df["subscriberCount"].replace(0, np.nan, inplace=True)
    df["friendCount"].replace(0, np.nan, inplace=True)
    
    df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
    df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
    df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
    df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
    
    df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
    df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
        
    for col in df.columns:
        if df[col].dtype.name == "object":
            df[col] = df[col].apply(str_normalize).astype("category")
            
    return df


def transform(df):
    df = process_gender(df)
    df = process_location(df)
    df = process_diaChi_maCv(df)
    df = process_misc(df)
    return df
df_new_fe = transform(df_fe)
df_new_fe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73411 entries, 0 to 20380
Columns: 248 entries, id to gender
dtypes: category(112), float64(133), int64(3)
memory usage: 88.5 MB


In [7]:
df_new_fe = df_new_fe.drop(["gioiTinh","info_social_sex"],1)
def convert_int(s):
  try:
    return int(s)
  except:
    return np.nan

for col in new_col + new_col_1 + new_col_2:
  df_new_fe[col] = df_new_fe[col].apply(convert_int)


drop_cat = ["Field_48", "Field_45", "Field_68", "Field_18", "Field_55", 
            
            "Field_54", "Field_36", "currentLocationLocationId", "homeTownLocationId", 

            "data.basic_info.locale", "currentLocationCity", "currentLocationCountry", 

            "currentLocationName", "currentLocationState", "homeTownCity", "homeTownName", 

            "homeTownCountry", "homeTownState"



            ]

df_new_fe = df_new_fe.drop(drop_cat, 1)


for i in list(df_new_fe.select_dtypes(['category'])):
  print(i + ": num of cat:    " + str(len(df_new_fe[i].unique())))

df_new_fe["Field_34month"] = df_new_fe["Field_34month"].apply(convert_int)

df_new_fe["Field_34year"] = df_new_fe["Field_34year"].apply(convert_int)

#create one hot encoding for categorical column
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res)

for col in list(df_new_fe.select_dtypes(['category'])):
  df_new_fe = encode_and_bind(df_new_fe, col)

Field_4: num of cat:    3
Field_12: num of cat:    8
diaChi: num of cat:    63
Field_46: num of cat:    9
Field_49: num of cat:    64
Field_56: num of cat:    10
Field_61: num of cat:    8
Field_65: num of cat:    11
Field_66: num of cat:    9
maCv: num of cat:    18
brief: num of cat:    21
Field_34year: num of cat:    15
Field_34month: num of cat:    13
gender: num of cat:    3


In [8]:
y_label = df_train["label"]
train_fe = df_new_fe[df_new_fe["id"] < df_train.shape[0]]
test_fe = df_new_fe[df_new_fe["id"] >= df_train.shape[0]]

print(train_fe.shape)
print(test_fe.shape)

(53030, 443)
(20381, 443)


In [9]:
def gini(y_true, y_score):
    return roc_auc_score(y_true, y_score)*2 - 1

def lgb_gini(y_pred, dataset_true):
    y_true = dataset_true.get_label()
    return 'gini', gini(y_true, y_pred), True

NUM_BOOST_ROUND = 1000

lgbm_param = {'objective':'binary',
              'boosting_type': 'gbdt',
              'metric' : 'auc',
              'learning_rate': 0.015,
              "bagging_freq": 1,
              "bagging_fraction" : 0.25,
              'tree_learner': 'serial',
              'reg_lambda': 2,
              'reg_alpha': 1,              
              "feature_fraction": 0.15,
              'num_leaves': 16,
              'max_depth': 8,
              'random_state': 16111997,
            }

seeds = np.random.randint(0, 10000, 3)
preds = 0    
feature_important = None
avg_train_gini = 0
avg_val_gini = 0

for s in seeds:
    skf = StratifiedKFold(n_splits=5, random_state=s, shuffle=True)        
    lgbm_param['random_state'] = s    
    seed_train_gini = 0
    seed_val_gini = 0
    for i, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y_label)), y_label)):                
        X_train, X_val = train_fe.iloc[train_idx].drop(["id"], 1), train_fe.iloc[val_idx].drop(["id"], 1)                
        y_train, y_val = y_label[train_idx], y_label[val_idx]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval  = lgb.Dataset(X_val, y_val)

        evals_result = {} 
        model = lgb.train(lgbm_param,
                    lgb_train,
                    num_boost_round=NUM_BOOST_ROUND,  
                    early_stopping_rounds=50,
                    feval=lgb_gini,
                    verbose_eval=False,
                    evals_result=evals_result,
                    valid_sets=[lgb_train, lgb_eval])

        seed_train_gini += model.best_score["training"]["gini"] / skf.n_splits
        seed_val_gini += model.best_score["valid_1"]["gini"] / skf.n_splits

        avg_train_gini += model.best_score["training"]["gini"] / (len(seeds) * skf.n_splits)
        avg_val_gini += model.best_score["valid_1"]["gini"] / (len(seeds) * skf.n_splits)

        if feature_important is None:
            feature_important = model.feature_importance() / (len(seeds) * skf.n_splits)
        else:
            feature_important += model.feature_importance() / (len(seeds) * skf.n_splits)        

        pred = model.predict(test_fe.drop(["id"], 1))
        preds += pred / (skf.n_splits * len(seeds))
        
        print("Fold {}: {}/{}".format(i, model.best_score["training"]["gini"], model.best_score["valid_1"]["gini"]))
    print("Seed {}: {}/{}".format(s, seed_train_gini, seed_val_gini))

print("-" * 30)
print("Avg train gini: {}".format(avg_train_gini))
print("Avg valid gini: {}".format(avg_val_gini))
print("=" * 30)

Fold 0: 0.5646257038000715/0.47548980225543147
Fold 1: 0.5591914582820059/0.49285450287238364
Fold 2: 0.5655891238904167/0.46915911822830636
Fold 3: 0.5621448599884715/0.49404792152447685
Fold 4: 0.5646296948460001/0.4753102093595063
Seed 635: 0.5632361681613931/0.4813723108480209
Fold 0: 0.5636746886835311/0.47948827267589356
Fold 1: 0.5618800270098214/0.4928819443945205
Fold 2: 0.5628572841157757/0.48883210155269285
Fold 3: 0.564275960328342/0.47159759324705197
Fold 4: 0.5627996715436403/0.4754824172349519
Seed 485: 0.5630975263362221/0.4816564658210222
Fold 0: 0.5634644123716406/0.47349563963040353
Fold 1: 0.5609379587592005/0.4922043688464286
Fold 2: 0.5616719157137104/0.47584223803195314
Fold 3: 0.5609811219750089/0.48646679023293804
Fold 4: 0.5638327748469694/0.47192791851576854
Seed 7673: 0.562177636733306/0.47998739105149835
------------------------------
Avg train gini: 0.562837110410307
Avg valid gini: 0.48100538924018044


In [10]:
# eliminte column with the feature importance less than 5
features = train_fe.columns.tolist()
features.remove("id")
df_imp = pd.DataFrame(data = {'col' : features , 'imp' : feature_important})
df_imp = df_imp.sort_values(by='imp', ascending=False).reset_index(drop=True)
drop_index = [i for i in range(len(df_imp['imp'])) if df_imp['imp'][i] <= 5]
drop_again = df_imp['col'][drop_index]

train_fe = train_fe.drop(drop_again,1)
test_fe = test_fe.drop(drop_again,1)

missing_col = []
for col in list(train_fe):
  if train_fe[col].isna().sum()/ 53030 >= 0.75:
    missing_col.append(col)

train_fe = train_fe.drop(missing_col,1)
test_fe = test_fe.drop(missing_col,1)

**Apply catboost**

In [11]:
!pip install catboost
!pip install category_encoders

#from datetime import datetime
#from unidecode import unidecode
#from itertools import combinations

#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import cross_val_score, StratifiedKFold
#from sklearn.metrics import roc_auc_score

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import category_encoders as ce


gini, feature_importance_df = {}, pd.DataFrame()

TRAIN, TEST = train_fe.drop(['id'], axis=1), test_fe.drop(['id'], axis=1)
LABEL = df_train['label']
preds, oof_preds = np.zeros(TRAIN.shape[0]), {}

cv = StratifiedKFold(n_splits=5, shuffle=True)
for i, (train_idx, val_idx) in enumerate(cv.split(TRAIN, LABEL)):
    X_train, y_train = TRAIN.iloc[train_idx], LABEL.iloc[train_idx]
    X_val, y_val = TRAIN.iloc[val_idx], LABEL.iloc[val_idx]

    gbm = CatBoostClassifier(eval_metric='AUC', 
                             use_best_model=True,
                             iterations=1000, 
                             learning_rate=0.1, 
                             random_seed=42).fit(X_train, y_train, 
                                                 #cat_features=set(cat_features),
                                                 eval_set=(X_val, y_val), verbose=500)

    y_pred = gbm.predict(X_val)
    y_pred_proba = gbm.predict_proba(X_val)[:, 1]
        
    preds[val_idx] = y_pred_proba
    oof_preds[f'F{i+1}'] = gbm.predict_proba(TEST)[:, 1]
    
    gini[f'F{i+1}'] = 2 * roc_auc_score(y_val, y_pred_proba) - 1
    
    # For create feature importances
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = TRAIN.columns
    fold_importance_df["importance"] = gbm.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    print('Fold %2d GINI : %.5f' % (i + 1, 2*roc_auc_score(y_val, y_pred_proba) - 1))
    
# Resulting
roc_auc = roc_auc_score(LABEL, preds)
print('Avg GINI score:', 2*roc_auc - 1)

result = np.array(list(gini.values()))
print('GINI: {:.5f} +- {:.5f}'.format(result.mean(), result.std()))

0:	test: 0.6482310	best: 0.6482310 (0)	total: 98.2ms	remaining: 1m 38s
500:	test: 0.7492735	best: 0.7503388 (410)	total: 18.3s	remaining: 18.2s
999:	test: 0.7462329	best: 0.7503388 (410)	total: 35.8s	remaining: 0us

bestTest = 0.7503387953
bestIteration = 410

Shrink model to first 411 iterations.
Fold  1 GINI : 0.50068
0:	test: 0.6534870	best: 0.6534870 (0)	total: 42.1ms	remaining: 42.1s
500:	test: 0.7518048	best: 0.7534455 (327)	total: 17.6s	remaining: 17.6s
999:	test: 0.7491246	best: 0.7534455 (327)	total: 34.9s	remaining: 0us

bestTest = 0.753445544
bestIteration = 327

Shrink model to first 328 iterations.
Fold  2 GINI : 0.50689
0:	test: 0.6483985	best: 0.6483985 (0)	total: 37.9ms	remaining: 37.9s
500:	test: 0.7486143	best: 0.7487578 (341)	total: 18.5s	remaining: 18.4s
999:	test: 0.7488390	best: 0.7498023 (787)	total: 36s	remaining: 0us

bestTest = 0.7498022506
bestIteration = 787

Shrink model to first 788 iterations.
Fold  3 GINI : 0.49960
0:	test: 0.6448674	best: 0.6448674 (0)	

**Submisson catboost**

In [12]:

df_test_c = df_test.copy()
# save to csv file
df_test_c['label'] = pd.DataFrame(oof_preds).mean(axis=1).values
name = pd.Timestamp.now().strftime('%Y%m%d_%H_%M')
df_test_c[['id', 'label']].to_csv('submisson_30.csv', index=False)

**Apply LGBM again**

In [13]:

from multiprocessing import cpu_count

def gini(y_true, y_score):
    return roc_auc_score(y_true, y_score)*2 - 1

def lgb_gini(y_pred, dataset_true):
    y_true = dataset_true.get_label()
    return 'gini', gini(y_true, y_pred), True

def model_lgb_gini(y_true, y_pred):
    return 'gini', gini(y_true, y_pred), True

def xgb_gini(y_pred, dataset_true):
    y_true = dataset_true.get_label()
    return 'gini', float(gini(y_true, y_pred))



def get_feat_imp(X, y):
    model_lgb = lgb.LGBMClassifier(
        boosting_type='gbdt', 
        num_leaves=31, 
        max_depth= -1, 
        learning_rate=0.1, 
        n_estimators=1000, 
        objective='binary',
        metric='auc',
        min_data_in_leaf=10, 
        lambda_l1=9.573264707286953, 
        lambda_l2=0.26292036067115865,
        feature_fraction=0.5,
        bagging_fraction=1.0,
        bagging_freq=0,
        n_jobs= cpu_count()
        )
    folds = StratifiedKFold(n_splits=5, shuffle=False, random_state=42)
    oof = np.zeros(len(X))
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print("Fold {}".format(fold_))
        X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
        model_lgb.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, eval_metric=model_lgb_gini, verbose=50)
        oof[val_idx] = model_lgb.predict(X_val, num_iteration=model_lgb.best_iteration_)
    print("Best AUC score:", model_lgb.best_score_['valid_0']['auc'])
    print("Best Gini score:", model_lgb.best_score_['valid_0']['gini'])
    print("No. data's features: ", model_lgb.n_features_)
    feature_imp = pd.DataFrame(sorted(zip(model_lgb.feature_importances_,X.columns)), columns=['Value','Feature'])
    return feature_imp




In [14]:
def run_model(train_df, test_df):
    features = [c for c in train_df.columns if c not in ['id', 'label']]
    label = train_df['label']
    num_round = 1000000
    folds = StratifiedKFold(n_splits=5, shuffle=False, random_state=42)
    oof = np.zeros(len(train_df))
    predictions = np.zeros(len(test_df))
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, label.values)):
            print("Fold {}".format(fold_))
            trn_data = lgb.Dataset(
                train_df.iloc[trn_idx][features], label=label.iloc[trn_idx])
            val_data = lgb.Dataset(
                train_df.iloc[val_idx][features], label=label.iloc[val_idx])
            clf = lgb.train(params, trn_data, num_round, valid_sets=[trn_data, val_data],
                            verbose_eval = 1000, feval = lgb_gini)
            oof[val_idx] = clf.predict(
                train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
            
            predictions += clf.predict(test_df[features],
                                       num_iteration=clf.best_iteration) / folds.n_splits
    print("CV score: {:<8.5f}".format(gini(label, oof)))
    return predictions


params = {
    'objective': 'binary',     
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'early_stopping': 2500,
    'lambda_l1': 9.573264707286953, 
    'lambda_l2': 0.26292036067115865, 
    'num_leaves': 31, 
    'feature_fraction': 0.5, 
    'bagging_fraction': 1.0, 
    'bagging_freq': 0, 
    'min_child_samples': 10,
    'verbose': -1
    }


y_train = df_train["label"]
X_train = train_fe.drop(columns=["id"], axis=1)
X_test = test_fe.drop(columns=["id"], axis=1)
X_test_id = test_fe["id"]

feature_imp = get_feat_imp(X_train, y_train)
imp = feature_imp[feature_imp['Value'] > 0]['Feature']
imp = imp.append(pd.Series(['label'], index=[0]), ignore_index=True)
print("No. training features:", len(imp))

train_fe['label'] = df_train['label']
test_fe['label'] = np.array([-1 for i in range(len(df_test['id']))])

predictions = run_model(train_fe[imp], test_fe[imp])
print(predictions)

result = pd.DataFrame({'id':X_test_id, 'label':predictions})
result.to_csv("submission_31.csv", index=False)


Fold 0
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.734483	valid_0's gini: 0.468966
[100]	valid_0's auc: 0.738606	valid_0's gini: 0.477212
[150]	valid_0's auc: 0.740195	valid_0's gini: 0.48039
[200]	valid_0's auc: 0.739615	valid_0's gini: 0.47923
Early stopping, best iteration is:
[149]	valid_0's auc: 0.740245	valid_0's gini: 0.480491
Fold 1
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.739432	valid_0's gini: 0.478863
[100]	valid_0's auc: 0.743521	valid_0's gini: 0.487041
[150]	valid_0's auc: 0.746134	valid_0's gini: 0.492268
[200]	valid_0's auc: 0.745281	valid_0's gini: 0.490561
Early stopping, best iteration is:
[148]	valid_0's auc: 0.746244	valid_0's gini: 0.492489
Fold 2
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's auc: 0.747832	valid_0's gini: 0.495665
[100]	valid_0's auc: 0.752052	valid_0's gini: 0.504105
[150]	valid_0's auc: 0.754465	valid_0's gini: 0.50893
[200]	valid

**Taking average of results**

In [15]:
model_2 = result.copy()

model_1 = df_test_c[["id","label"]]

model_avg = pd.DataFrame({"id": model_1["id"].values})

model_avg["label"] = [(model_1["label"][i]+ model_2["label"][i])/2 for i in range(len(model_1["id"]))]
model_avg.to_csv("submission_32.csv", index=False)